In [1]:
import numpy as np
import pandas as pd

In [2]:
fname = '가 나 다 라 마 바-1 바 사 사-1 사-2 아 자 차 카 타 파 하 A B C D E F G H I J K L M N O P Q R S T U V W X Y Z'.split()

In [3]:
df = pd.DataFrame()
for fn in fname:
    try:
        tmp = pd.read_csv(f'../static/data/movie{fn}.csv')
    except:
        continue
    df = pd.concat([df, tmp])
df.shape

(11273, 10)

In [4]:
df.head(1)

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis
0,20231523,그 여름 (The Summer),애니메이션(2),"장편 | 예술,독립 영화 | 애니메이션, 드라마, 멜로/로맨스 | 60분 57초 |...",2023-06-07,2023년,한지원,윤아영 | 송하림,https://www.kobis.or.kr/common/mast/movie/2023...,"""서로의 몸은 차라리 꽃잎과 물결에 가까웠다""\r\n갈색 눈동자를 가진 평범한 학생..."


In [5]:
df.set_index('code', inplace=True)
df.reset_index(inplace=True)
df.fillna('', inplace=True)
print(df.shape)
df.head(1)


(11273, 10)


,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis
0,20231523,그 여름 (The Summer),애니메이션(2),"장편 | 예술,독립 영화 | 애니메이션, 드라마, 멜로/로맨스 | 60분 57초 |...",2023-06-07,2023년,한지원,윤아영 | 송하림,https://www.kobis.or.kr/common/mast/movie/2023...,"""서로의 몸은 차라리 꽃잎과 물결에 가까웠다""\r\n갈색 눈동자를 가진 평범한 학생..."


In [6]:
df.isna().sum()

code               0
title              0
genre              0
summering          0
first_day          0
production_year    0
movie_director     0
star_actor         0
img                0
synopsis           0
dtype: int64

In [12]:
df.drop_duplicates(subset='code', inplace=True)

In [13]:
df.code.nunique(), df.shape

(11271, (11271, 10))

In [16]:
df[df.summering == ('장편 | 일반영화 | 공연')]

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis
3428,20193151,마리아 스투아르다 (Maria Stuarda ),공연,장편 | 일반영화 | 공연,해당정보없음,2020년,데이비드 맥비커,,https://www.kobis.or.kr/kobis/web/comm/images/...,스코틀랜드의 메리 스튜어트 여왕과 잉글랜드의 엘리자베스 여왕. 권력과 사랑을 놓고 ...


In [21]:
df.drop(df.index[3428], inplace=True)
df.shape

(11270, 10)

In [23]:
df.set_index('code', inplace=True)
df.reset_index(inplace=True)

In [24]:
m_type, m_kind, m_genre, m_time, m_rated, m_nation = [], [], [], [], [], []
for info in df.summering.values:
    infos = info.split('|')
    infos_len = len(infos)
    m_type.append(infos[0].strip())                     # 장편
    m_kind.append(infos[1].strip())                     # 일반영화, 예술영화..
    m_genre.append(infos[2].strip().replace(',', ''))   # 애니메이션, 드라마, 멜로
    m_nation.append(infos[-1].strip())                  # 한국
    
    # 요약 정보 개수가 다를 수 있다
    if infos_len == 6:
        m_time.append(infos[-3].strip())                # 70분
        m_rated.append(infos[-2].strip())
    elif infos_len == 5 :
        m_time.append(infos[-2].strip())                # 12세 이상 관람가
        m_rated.append('')
    elif infos_len == 4:
        m_rated.append('')
        m_time.append('')
    else:
        print(info)

len(m_type)

11270

In [25]:
df['m_type'] = m_type
df['m_kind'] = m_kind
df['m_genre'] = m_genre
df['m_nation'] = m_nation
df['m_time'] = m_time
df['m_rated'] = m_rated

In [26]:
df.head(1)

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis,m_type,m_kind,m_genre,m_nation,m_time,m_rated
0,20231523,그 여름 (The Summer),애니메이션(2),"장편 | 예술,독립 영화 | 애니메이션, 드라마, 멜로/로맨스 | 60분 57초 |...",2023-06-07,2023년,한지원,윤아영 | 송하림,https://www.kobis.or.kr/common/mast/movie/2023...,"""서로의 몸은 차라리 꽃잎과 물결에 가까웠다""\r\n갈색 눈동자를 가진 평범한 학생...",장편,"예술,독립 영화",애니메이션 드라마 멜로/로맨스,한국,60분 57초,12세이상관람가


In [27]:
df.tail(5)

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis,m_type,m_kind,m_genre,m_nation,m_time,m_rated
11265,20203281,XXX필름 연쇄살인 (Cry Havoc),액션(1),"장편 | 일반영화 | 액션, 범죄 | 85분 54초 | 미국",해당정보없음,2019년,르네 페르즈,,https://www.kobis.or.kr/kobis/web/comm/images/...,대저택의 소유자이자 FBI의 일순위 수배자 ‘보이어’는\nTV 프로그램 촬영을 빌미...,장편,일반영화,액션 범죄,미국,85분 54초,
11266,20188682,XXX 홀릭 극장판 : 한 여름 밤의 꿈 (xxxHOLiC the Movie: A ...,애니메이션(5),"장편 | 일반영화 | 애니메이션, 코미디, 드라마, 판타지, 미스터리, 스릴러 | ...",해당정보없음,2005년,미즈시마 츠토무,후쿠야마 준 | 나카이 카즈야,https://www.kobis.or.kr/kobis/web/comm/images/...,어떤 소원이든 그에 상응하는 대가만 지불하면 다 해결해 주는 마녀 이치하라 유코.\...,장편,일반영화,애니메이션 코미디 드라마 판타지 미스터리 스릴러,일본,60분 0초,
11267,20020049,YMCA야구단 (YMCA Baseball Team),드라마(1),"장편 | 일반영화 | 드라마, 코미디 | 104분 0초 | 전체관람가 | 한국",2002-10-02,2002년,김현석,송강호 | 김혜수(민정림) | 김주혁 | 황정민,https://www.kobis.or.kr/upload/up_img/cleansin...,글공부보다 운동을 더 좋아하는 선비 호창은 젊은 시절의 유일한 꿈이었던 과거제도가 ...,장편,일반영화,드라마 코미디,한국,104분 0초,전체관람가
11268,19828097,Z 특공대 (Attack Force Z),액션(2),"장편 | 일반영화 | 액션, 드라마, 전쟁 | 93분 0초 | 호주",해당정보없음,1982년,,,https://www.kobis.or.kr/kobis/web/comm/images/...,불굴의 정신과 명예를 위해 극비리에 차출된 비밀결사 어택포스 Z 특공대 23인이 투...,장편,일반영화,액션 드라마 전쟁,호주,93분 0초,
11269,20190105,Z 파이터스 (Zombie Fighters),액션(3),"장편 | 일반영화 | 액션, 스릴러, 공포(호러), 어드벤처 | 100분 0초 | 태국",해당정보없음,2017년,포 아르논,,https://www.kobis.or.kr/common/mast/movie/2019...,"폐쇄된 병원, 바이러스 그리고 괴생명체...\n생존확률 제로의 위험한 사투가 시작된...",장편,일반영화,액션 스릴러 공포(호러) 어드벤처,태국,100분 0초,


In [28]:
indexs = df[df.summering.str.contains('멜로/로맨스') & df.summering.str.contains('청소년관람불가')].index
indexs

Int64Index([], dtype='int64')

In [33]:

print(df[df.star_actor.str.find('[') >= 0][['star_actor']])

                                              star_actor
10883  김윤진(홍정혜) | 나문희(김문옥) | 강예원(강유미) | 이다희(공나영) | 장영...


In [13]:
# 감독 자리에 다른 값들이 들어갔다. 주연 배우 자리에는 데이터가 없고
# 잘못 처리했다. 일단 감독이 이상한 것은 지우고 테스트
indexs = df[df.movie_director.str.find('[') >= 0].index
indexs

Index([], dtype='int64')

In [142]:
df.drop(df.index[indexs], inplace=True)
df.set_index('code', inplace=True)
df.reset_index(inplace=True)

In [143]:
df.shape

(2746, 16)

##### 시놉시스 형태소 분석해 놓기

In [34]:
from konlpy.tag import Okt
okt = Okt()
stop_words = '''의 가 에 과 을 의 를 는 이 은 와 
            '''.split()

In [35]:
import re, string

morphs = []

for text in df.synopsis:
    text = re.sub('['+string.punctuation+'\r\n“”‘’…]', ' ', text)
    tokens = okt.morphs(text)
    morphs.append(' '.join([word for word in tokens if word not in stop_words]))

df['morphs'] = morphs


In [36]:
df.head(1)

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis,m_type,m_kind,m_genre,m_nation,m_time,m_rated,morphs
0,20231523,그 여름 (The Summer),애니메이션(2),"장편 | 예술,독립 영화 | 애니메이션, 드라마, 멜로/로맨스 | 60분 57초 |...",2023-06-07,2023년,한지원,윤아영 | 송하림,https://www.kobis.or.kr/common/mast/movie/2023...,"""서로의 몸은 차라리 꽃잎과 물결에 가까웠다""\r\n갈색 눈동자를 가진 평범한 학생...",장편,"예술,독립 영화",애니메이션 드라마 멜로/로맨스,한국,60분 57초,12세이상관람가,서로 몸 차라리 꽃잎 물결 가까웠다 갈색 눈동자 가진 평범한 학생 이경 여름 햇살 ...


In [37]:
df.to_csv('../static/data/movie.csv', index=False)

In [ ]:
for title in df.title:
    print(title)